In [1]:
import os
import shutil
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pickle  #파일저장에

In [2]:


#pip install tqdm
from tqdm import tqdm # 학습 진행 상황 시각화를 위해 추가

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from PIL import Image

In [3]:

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
# 이후 PyTorch, NumPy, TensorFlow 등을 import 합니다.
# 경고 무시 설정
warnings.filterwarnings('ignore')

# 1. 경로 설정 및 파라미터
original_dataset_dir = './data/cats_and_dogs/train'
"""
cats_and_dogs
   ㄴ train 

cats_and_dogs_small
   ㄴ train 
       ㄴcats
       ㄴdogs 
   ㄴ test 
       ㄴcats
       ㄴdogs 
   ㄴ validation 
        ㄴcats
       ㄴdogs 

"""
base_dir = './data/cats_and_dogs_small'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test') #베이스 아래 3가지 폴더 생성

model_save_path_pth = 'cats_and_dogs_model.pth'
history_filepath = 'cats_and_dogs_history.pkl'

batch_size = 16
img_height = 180
img_width = 180
num_epochs = 30 # 예시로 epoch 수를 30으로 설정했습니다.
learning_rate = 0.001 # learning_rate 추가

# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 중인 디바이스: {device}")

# 이미지 복사 함수 
def ImageCopyMove():
    #경로가 있는지 확인해본다  경로가 있으면 True 없으면 False 
    if os.path.exists(base_dir): #베이스 아래
        #shutil :shell util - 명령어 해석기 
        shutil.rmtree(base_dir, ignore_errors=True)
    os.makedirs(train_dir)  #디렉토리 생성 3개
    os.makedirs(validation_dir)
    os.makedirs(test_dir)
    
    #3개의 폴더에 고양이,강아지 파일 생성 하나씩
    train_cats_dir = os.path.join(train_dir, 'cats')
    train_dogs_dir = os.path.join(train_dir, 'dogs')
    validation_cats_dir = os.path.join(validation_dir, 'cats')
    validation_dogs_dir = os.path.join(validation_dir, 'dogs')
    test_cats_dir = os.path.join(test_dir, 'cats')
    test_dogs_dir = os.path.join(test_dir, 'dogs')

    os.makedirs(train_cats_dir)
    os.makedirs(train_dogs_dir)
    os.makedirs(validation_cats_dir)
    os.makedirs(validation_dogs_dir)
    os.makedirs(test_cats_dir)
    os.makedirs(test_dogs_dir)
    
    #["cat.0.jpg", "cat.1.jpg",..... ]
    fnames = ['cat.{}.jpg'.format(i) for i in range(1000)] #이름 지정 트레인
    for fname in fnames:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(train_cats_dir, fname)
        shutil.copyfile(src, dst)
        
    fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)] #검증
    for fname in fnames:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(validation_cats_dir, fname)
        shutil.copyfile(src, dst)
    
    fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)] #테스트
    for fname in fnames: 
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(test_cats_dir, fname)
        shutil.copyfile(src, dst)
    

    # ---강아지로
    
    fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(train_dogs_dir, fname)
        shutil.copyfile(src, dst)
    
    fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(validation_dogs_dir, fname)
        shutil.copyfile(src, dst)
    
    fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
    for fname in fnames:
        src = os.path.join(original_dataset_dir, fname)
        dst = os.path.join(test_dogs_dir, fname)
        shutil.copyfile(src, dst)

    print("이미지 복사 및 폴더 생성 완료!")


ImageCopyMove()  

#꽃분류_cnn 처럼 cnn분류하기 

사용 중인 디바이스: cpu
이미지 복사 및 폴더 생성 완료!


In [4]:
import numpy as np
import os
import random

from PIL import Image as pilimg
import imghdr
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [5]:
def makeDate(folder,label,isTrain):
    base_dir = "./data/cats_and_dogs_small"
    if isTrain == 'train':
        path = os.path.join(base_dir,"train",folder)
    elif isTrain == 'test':
        path = os.path.join(base_dir,"test",folder)
    else:
        raise ValueError('train,test,val,validation 중 하나여야 합니다.')
    
    data = []
    labels = []
    
    i = 1
    for filename in os.listdir(path):
        if i % 100 == 0:
            print(i, filename)
        i += 1
        
        file_path = os.path.join(path, filename)
        try:
            kind = imghdr.what(path + "/" + filename)
            if kind in ['gif',"png","jpeg","jpg"]:
                img = pilimg.open(path + '/' + filename).convert('RGB')
                resize_img = img.resize((150,150))
                pixel = np.array(resize_img)

                if pixel.shape == (150, 150, 3):
                    data.append(pixel)
                    labels.append(label)

        except Exception as e:
            print(f"{filename} error: {e}")

    data = np.array(data)       # 리스트 → 넘파이 배열
    labels = np.array(labels)

    out_name = f"imagedata{label}_{isTrain}.npz"
    np.savez(out_name, data=data, targets=labels)
    print(out_name, '파일 저장 완료')

In [6]:
#첫 1회
def dataCreate():
    cutes = ['dog','cat']
    label = 0
    for cute in cutes:
        print(cute + '작업중....')
        makeDate(cute,label,'train')
        makeDate(cute,label,'test')
        label += 1

In [7]:
def load_and_concat_data(mode, num_labels=2):
    datas = []
    targets = []
    for label in range(num_labels):
        
        file_name = f"imagedata{label}_{mode}.npz"
        npz_file =np.load(file_name)

        datas.append(npz_file["data"])
        targets.append(npz_file["targets"])

    return np.concatenate(datas, axis= 0), np.concatenate(targets, axis = 0)


In [9]:
train_Data_np,train_Target_np = load_and_concat_data('train',num_labels=2)
test_Data_np,test_Target_np = load_and_concat_data('test',num_labels=2)

In [ ]:

#a( (1,2,3) )
# 데이터 형태 변환 (PyTorch에 맞게)
# PyTorch는 `channels_first` (C, H, W) 형태를 선호합니다.
# 데이터 정규화: 0-255 -> 0.0-1.0
# numpy => torch.tensor => dataset => dataloader
#permute(0, 3, 1, 2) numpy => pytorch tensor 로 바꿀때 사용하는 함수 
#permute - 차원순서 0 1 2 3 

In [ ]:
trainData_tensor = (torch.from_numpy(train_Data_np)
                    .float()
                    .permute(0,3, 1, 2)
                    / 255.0)

testData_tensor = (torch.from_numpy(test_Data_np)
                   .float()
                   .permute(0,3, 1, 2)
                    / 255.0 )

#trainData_tensor = torch.from_numpy(trainData_np).float().permute(0, 3, 1, 2) / 255.0 .premute붙여서 오류


In [24]:
trainTarget_tensor = torch.from_numpy(train_Target_np).long()
testTarget_tensor = torch.from_numpy(test_Target_np).long()

In [27]:
print(f"Train Data Shape: {trainData_tensor.shape}")
print(f"Test Data Shape: {testData_tensor.shape}")

Train Data Shape: torch.Size([1278, 3, 150, 150])
Test Data Shape: torch.Size([543, 3, 150, 150])


In [29]:
train_dataset = TensorDataset(trainData_tensor,trainTarget_tensor)
test_dataset = TensorDataset(testData_tensor,testTarget_tensor)

train_loader = DataLoader(train_dataset,batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=64, shuffle= False)

In [57]:
#Train Data Shape: torch.Size([1278, 3, 150, 150])
#Test Data Shape: torch.Size([543, 3, 150, 150])

class CuteLeNN(nn.Module):
    def __init__(self):
        super(CuteLeNN,self).__init__()

        self.conv1 = nn.Conv2d( 3 ,32,kernel_size=3, stride=1,padding=1)
        self.pool1 = nn.MaxPool2d( kernel_size=2,stride=2)

        self.conv2 = nn.Conv2d( 32 ,64,kernel_size=3, stride=1,padding=1)
        self.pool2 = nn.MaxPool2d( kernel_size=2,stride=2)

        self.fc_input_size = 64 * 37* 37

        self.fc1 = nn.Linear(self.fc_input_size,256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256,64)
        self.fc3 = nn.Linear(64,32)
        self.fc4 = nn.Linear(32,2) #2개
    def forward(self,x):

        x = self.relu(self.conv1(x))
        x = self.pool1(x)

        x = self.relu(self.conv2(x))
        x = self.pool2(x)
        
        x = x.view(-1,self.fc_input_size)

        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)

        return x




In [58]:
model = CuteLeNN()
print(model)

CuteLeNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=87616, out_features=256, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=2, bias=True)
)


In [59]:
criterion =nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [62]:
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs,labels) in enumerate(train_loader):

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs,labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch {epoch + 1}/{epochs},  Loss: {running_loss/len(train_loader):.4f}')
    

Epoch 1/10,  Loss: 0.6903
Epoch 2/10,  Loss: 0.6893
Epoch 3/10,  Loss: 0.6887
Epoch 4/10,  Loss: 0.6882
Epoch 5/10,  Loss: 0.6878
Epoch 6/10,  Loss: 0.6875
Epoch 7/10,  Loss: 0.6872
Epoch 8/10,  Loss: 0.6868
Epoch 9/10,  Loss: 0.6863
Epoch 10/10,  Loss: 0.6860


In [63]:
model.eval()
train_correct = 0
train_total = 0
test_correct = 0
test_total = 0

In [64]:
with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model(inputs)
        _,predicted = torch.max(outputs.data,1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
    
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _,predicted = torch.max(outputs.data,1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

train_acc = 100 * train_correct / train_total
test_acc = 100 * test_correct / test_total

In [65]:
print(f'\n훈련셋 정확도: {train_acc:.2f}%')
print(f'테스트셋 정확도: {test_acc:.2f}%')


훈련셋 정확도: 55.24%
테스트셋 정확도: 63.72%
